In [4]:
import numpy as np
import pandas as pd
import pickle

In [5]:
from hmmlearn import hmm

In [6]:
np.random.seed(42)

In [8]:
Fraud_Data = pd.read_csv('fraudTrain.csv')
print ('Data imported')

Data imported


In [10]:
Fraud_Data.head(10)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",3/9/1988,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,6/21/1978,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1/19/1962,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1/12/1967,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,3/28/1986,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
5,5,1/1/2019 0:04,4.767270e+15,"fraud_Stroman, Hudson and Erdman",gas_transport,94.63,Jennifer,Conner,F,4655 David Island,...,40.3750,-75.2045,2158,Transport planner,6/19/1961,189a841a0a8ba03058526bcfe566aab5,1325376248,40.653382,-76.152667,0
6,6,1/1/2019 0:04,3.007470e+13,fraud_Rowe-Vandervort,grocery_net,44.54,Kelsey,Richards,F,889 Sarah Station Suite 624,...,37.9931,-100.9893,2691,Arboriculturist,8/16/1993,83ec1cc84142af6e2acf10c44949e720,1325376282,37.162705,-100.153370,0
7,7,1/1/2019 0:05,6.011360e+15,fraud_Corwin-Collins,gas_transport,71.65,Steven,Williams,M,231 Flores Pass Suite 720,...,38.8432,-78.6003,6018,"Designer, multimedia",8/21/1947,6d294ed2cc447d2c71c7171a3d54967c,1325376308,38.948089,-78.540296,0
8,8,1/1/2019 0:05,4.922710e+15,fraud_Herzog Ltd,misc_pos,4.27,Heather,Chase,F,6888 Hicks Stream Suite 954,...,40.3359,-79.6607,1472,Public affairs consultant,3/7/1941,fc28024ce480f8ef21a32d64c93a29f5,1325376318,40.351813,-79.958146,0
9,9,1/1/2019 0:06,2.720830e+15,"fraud_Schoen, Kuphal and Nitzsche",grocery_pos,198.39,Melissa,Aguilar,F,21326 Taylor Squares Suite 708,...,36.5220,-87.3490,151785,Pathologist,3/28/1974,3b9014ea8fb80bd65de0b1463b00b00e,1325376361,37.179198,-87.485381,0


In [6]:
df=Fraud_Data[['category', 'amt','gender', 'city', 'state', 'lat','long', 'job','is_fraud']]
df[18:19]

,category,amt,gender,city,state,lat,long,job,is_fraud
18,food_dining,63.07,M,Harborcreek,PA,42.1767,-79.9416,Event organiser,0


In [7]:
from sklearn.utils import resample
df_majority=df[df.is_fraud==0]
df_minority=df[df.is_fraud==1]
print(df_majority.shape,df_minority.shape)

(1042569, 9) (6006, 9)


In [8]:
df_minority_upsampled=resample(df_minority,n_samples=100000, random_state=123)
mydata3=pd.concat([df_majority[:100000],df_minority_upsampled])

print(df_minority_upsampled.shape,mydata3.shape)

(100000, 9) (200000, 9)


In [9]:
mydata3['is_fraud'].value_counts()

0    100000
1    100000
Name: is_fraud, dtype: int64

In [10]:
mydata3

,category,amt,gender,city,state,lat,long,job,is_fraud
0,misc_net,4.97,F,Moravian Falls,NC,36.0788,-81.1781,"Psychologist, counselling",0
1,grocery_pos,107.23,F,Orient,WA,48.8878,-118.2105,Special educational needs teacher,0
2,entertainment,220.11,M,Malad City,ID,42.1808,-112.2620,Nature conservation officer,0
3,gas_transport,45.00,M,Boulder,MT,46.2306,-112.1138,Patent attorney,0
4,misc_pos,41.96,M,Doe Hill,VA,38.4207,-79.4629,Dance movement psychotherapist,0
...,...,...,...,...,...,...,...,...,...
955470,shopping_net,988.84,F,Walnut Ridge,AR,36.0244,-90.9288,"Engineer, control and instrumentation",1
157489,misc_pos,6.33,F,Coulee Dam,WA,48.0379,-118.9419,"Pilot, airline",1
292895,grocery_pos,321.38,M,Sontag,MS,31.6453,-90.1801,"Librarian, academic",1
233813,gas_transport,8.99,F,Pembroke Township,IL,41.0646,-87.5917,"Psychotherapist, child",1


In [11]:
mydata3["gender"].unique()

array(['F', 'M'], dtype=object)

In [12]:
from sklearn.preprocessing import LabelEncoder 

le1 = LabelEncoder()
cut_lbl = le1.fit(mydata3["category"])
mydata3["category_lbl"] = le1.transform(mydata3["category"])


le2 = LabelEncoder()
gen_lbl = le2.fit(mydata3["gender"])
mydata3["gender_lbl"] = le2.transform(mydata3["gender"])

le3 = LabelEncoder()
cit_lbl = le3.fit(mydata3["city"])
mydata3["city_lbl"] = le3.transform(mydata3["city"])

le4 = LabelEncoder()
sta_lbl = le4.fit(mydata3["state"])
mydata3["state_lbl"] = le4.transform(mydata3["state"])

le5 = LabelEncoder()
jo_lbl = le5.fit(mydata3["job"])
mydata3["job_lbl"] = le5.transform(mydata3["job"])

In [13]:
le3.inverse_transform([339]) #city

array(['High Rolls Mountain Park'], dtype=object)

In [14]:
le4.inverse_transform([4]) #state

array(['CA'], dtype=object)

In [15]:
le5.inverse_transform([274]) #job

array(['Location manager'], dtype=object)

In [16]:
#gender = male
# amount = 2 watch

In [17]:
mydata3

,category,amt,gender,city,state,lat,long,job,is_fraud,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl
0,misc_net,4.97,F,Moravian Falls,NC,36.0788,-81.1781,"Psychologist, counselling",0,8,0,517,27,370
1,grocery_pos,107.23,F,Orient,WA,48.8878,-118.2105,Special educational needs teacher,0,4,0,600,47,428
2,entertainment,220.11,M,Malad City,ID,42.1808,-112.2620,Nature conservation officer,0,0,1,459,13,307
3,gas_transport,45.00,M,Boulder,MT,46.2306,-112.1138,Patent attorney,0,2,1,83,26,328
4,misc_pos,41.96,M,Doe Hill,VA,38.4207,-79.4629,Dance movement psychotherapist,0,9,1,212,45,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955470,shopping_net,988.84,F,Walnut Ridge,AR,36.0244,-90.9288,"Engineer, control and instrumentation",1,11,0,821,2,162
157489,misc_pos,6.33,F,Coulee Dam,WA,48.0379,-118.9419,"Pilot, airline",1,9,0,173,47,341
292895,grocery_pos,321.38,M,Sontag,MS,31.6453,-90.1801,"Librarian, academic",1,4,1,745,25,270
233813,gas_transport,8.99,F,Pembroke Township,IL,41.0646,-87.5917,"Psychotherapist, child",1,2,0,626,14,374


In [18]:
dff = mydata3.drop(['category','gender', 'city', 'state', 'job'],axis=1)
dff

,amt,lat,long,is_fraud,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl
0,4.97,36.0788,-81.1781,0,8,0,517,27,370
1,107.23,48.8878,-118.2105,0,4,0,600,47,428
2,220.11,42.1808,-112.2620,0,0,1,459,13,307
3,45.00,46.2306,-112.1138,0,2,1,83,26,328
4,41.96,38.4207,-79.4629,0,9,1,212,45,116
...,...,...,...,...,...,...,...,...,...
955470,988.84,36.0244,-90.9288,1,11,0,821,2,162
157489,6.33,48.0379,-118.9419,1,9,0,173,47,341
292895,321.38,31.6453,-90.1801,1,4,1,745,25,270
233813,8.99,41.0646,-87.5917,1,2,0,626,14,374


In [19]:
cat_list = list(dff["category_lbl"].unique())
cat_list

[8, 4, 0, 2, 9, 3, 11, 12, 1, 10, 5, 13, 7, 6]

In [20]:
# for i in cat_list:
print(le1.inverse_transform(cat_list))
#     break

['misc_net' 'grocery_pos' 'entertainment' 'gas_transport' 'misc_pos'
 'grocery_net' 'shopping_net' 'shopping_pos' 'food_dining' 'personal_care'
 'health_fitness' 'travel' 'kids_pets' 'home']


In [21]:
cat_list

[8, 4, 0, 2, 9, 3, 11, 12, 1, 10, 5, 13, 7, 6]

In [22]:
x = dff.drop("is_fraud",axis=1)#Independent Variable 
y = dff["is_fraud"]# Dependent Variabl

In [23]:
from sklearn import preprocessing
# x = preprocessing.normalize(x)
# x = preprocessing.scale(x)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)

In [24]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100)  
  
clf.fit(x_train, y_train)
  
y_pred = clf.predict(x_test)
  
from sklearn import metrics  
print()
  
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.9959


In [36]:
filename = 'random_forest.sav'
pickle.dump(clf, open(filename, 'wb'))

In [11]:
filename='random_forest.sav'
loaded_model_nb = pickle.load(open(filename, 'rb'))

In [25]:
y_pred

array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [26]:
y_test.value_counts()

1    10039
0     9961
Name: is_fraud, dtype: int64

In [27]:
final_testing = pd.concat([x_test,y_test],axis=1)
final_testing

,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl,is_fraud
326811,50.65,43.2363,-78.3138,10,1,409,34,445,1
72977,18.99,43.7893,-75.4156,6,0,453,34,464,0
435960,292.00,45.1939,-112.0568,4,1,7,26,80,1
66086,74.86,38.4305,-90.3870,6,1,29,24,260,0
30379,14.94,38.3039,-85.4834,0,1,627,17,433,0
...,...,...,...,...,...,...,...,...,...
227269,11.61,29.0393,-95.4401,2,1,415,43,206,1
36327,67.12,43.0498,-78.8510,3,0,574,34,22,0
69412,985.29,41.1039,-72.3674,11,1,299,34,52,1
98677,5.05,41.2244,-86.6966,13,1,570,15,134,0


In [28]:
testing_data = final_testing.reset_index(drop=True)
testing_data

,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl,is_fraud
0,50.65,43.2363,-78.3138,10,1,409,34,445,1
1,18.99,43.7893,-75.4156,6,0,453,34,464,0
2,292.00,45.1939,-112.0568,4,1,7,26,80,1
3,74.86,38.4305,-90.3870,6,1,29,24,260,0
4,14.94,38.3039,-85.4834,0,1,627,17,433,0
...,...,...,...,...,...,...,...,...,...
19995,11.61,29.0393,-95.4401,2,1,415,43,206,1
19996,67.12,43.0498,-78.8510,3,0,574,34,22,0
19997,985.29,41.1039,-72.3674,11,1,299,34,52,1
19998,5.05,41.2244,-86.6966,13,1,570,15,134,0


# Fraud Detected

In [29]:
wrongoutput = testing_data[100:121]
# wrongoutput.to_csv('wrongoutput.csv')
wrongoutput

,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl,is_fraud
100,18.81,33.1194,-83.8235,10,0,387,10,463,1
101,41.46,48.8878,-118.2105,2,0,600,47,428,0
102,61.73,39.6321,-84.2675,2,0,496,35,255,0
103,34.75,39.8839,-75.4669,10,1,137,38,281,0
104,296.23,28.0600,-82.4079,4,0,778,9,487,1
105,58.65,40.2878,-74.0162,7,1,847,31,287,0
106,754.42,41.6125,-122.5258,12,1,305,4,443,1
107,1016.56,34.3470,-87.7154,11,0,630,1,153,1
108,32.65,38.3880,-79.9906,8,1,123,49,429,0
109,1096.53,29.3641,-98.4924,11,1,712,43,259,1


In [30]:
from scipy.spatial import distance

def proposedlogic(finaldff):
    threshold=0.50
    df1=pd.read_csv('wrongoutput.csv',sep=',',error_bad_lines=False)

    print(len(df1))
    print(df1)
    df=df1#.sample(n=1000,replace=True)
    X = df.iloc[:,:-1].values # We drop our "target" feature and use all the remaining features in our dataframe to train the model.
    y = df.iloc[:,-1].values

    X = df[['amt', 'lat', 'long', 'category_lbl', 'gender_lbl', 'city_lbl','state_lbl', 'job_lbl']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
    Y = df['is_fraud']
    X1 = X.iloc[:,:].values # We drop our "target" feature and use all the remaining features in our dataframe to train the model.
    finaldff = finaldff.apply(pd.to_numeric, errors='coerce')
    X2=finaldff.iloc[:,:].values
    
    #X2=[float(i) for i in X21]
    outexpect=[]
    overall=[]
    for row in X2:
        countis=0
        for row1 in X1:
            print(row1)
            y=row
            print('yis ',y)
            summation = 0
            print('check1')
            y_bar=row1
            print('y check ',y_bar)
            print('check11',y_bar,'second is',y)
            #if y!=y_bar:
            print('check2')
            n = len(y) #finding total number of items in list
            for i in range (0,n):  #looping through each element of the list
                difference = float(y[i]) - float(y_bar[i])  #finding the difference between observed and predicted value
                squared_difference = difference**2  #taking square of the differene 
                summation = summation + squared_difference  #taking a sum of all the differences
            MSE = summation/n  #dividing summation by total values to obtain average
            print("The Mean Square Error is: " , MSE)
            dst = distance.euclidean(y, y_bar)
            print("dst is: " , dst)
            if dst<0.02:
                outexpect.append(1.0)
            else:
                outexpect.append(0.0)
            if MSE<=0.5:
                outexpect.append(1.0)
            else:
                outexpect.append(0.0)
            #outexpect+=1
            #if len(outexpect)>len(X1):
            if len(outexpect)>10:
                break
    #outexpect.append(valis)
    #y_actu.append(expected)
    valmax=max(outexpect,key=outexpect.count)
    #outexpect
    overall.append(valmax)
    print('from the logic',valmax)
    return valmax
    #print(val,valis)


In [31]:
testing_data.columns

Index(['amt', 'lat', 'long', 'category_lbl', 'gender_lbl', 'city_lbl',
       'state_lbl', 'job_lbl', 'is_fraud'],
      dtype='object')

In [32]:
testdf=testing_data[testing_data.is_fraud==0]
testdf

,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl,is_fraud
1,18.99,43.7893,-75.4156,6,0,453,34,464,0
3,74.86,38.4305,-90.3870,6,1,29,24,260,0
4,14.94,38.3039,-85.4834,0,1,627,17,433,0
5,26.05,42.1939,-76.7361,7,0,92,34,373,0
9,86.27,30.2510,-92.5002,0,1,234,18,52,0
...,...,...,...,...,...,...,...,...,...
19993,77.45,42.0158,-73.2913,2,0,227,6,102,0
19994,2.83,39.9373,-79.1865,8,1,687,38,209,0
19996,67.12,43.0498,-78.8510,3,0,574,34,22,0
19998,5.05,41.2244,-86.6966,13,1,570,15,134,0


In [33]:
lst = testdf["is_fraud"]
testdf = testdf.drop("is_fraud",axis=1)
testdf

,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl
1,18.99,43.7893,-75.4156,6,0,453,34,464
3,74.86,38.4305,-90.3870,6,1,29,24,260
4,14.94,38.3039,-85.4834,0,1,627,17,433
5,26.05,42.1939,-76.7361,7,0,92,34,373
9,86.27,30.2510,-92.5002,0,1,234,18,52
...,...,...,...,...,...,...,...,...
19993,77.45,42.0158,-73.2913,2,0,227,6,102
19994,2.83,39.9373,-79.1865,8,1,687,38,209
19996,67.12,43.0498,-78.8510,3,0,574,34,22
19998,5.05,41.2244,-86.6966,13,1,570,15,134


In [35]:

nb=clf.predict(testdf)
output=nb[0]
if output==1:
    print(" According to HMM Classification Fraud Detected")
else:
    print(" According to HMM Classification No Fraud Detected")

 According to HMM Classification No Fraud Detected


In [81]:
testdf[1:2]

,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl
3,74.86,38.4305,-90.387,6,1,29,24,260


,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl
3,74.86,38.4305,-90.387,6,1,29,24,260


In [82]:
outputofproposed=proposedlogic(testdf[1:2])
outputofproposed

21
    Unnamed: 0      amt      lat      long  category_lbl  gender_lbl  \
0          100   366.72  41.4575  -74.1659             4           1   
1          101    41.46  48.8878 -118.2105             2           0   
2          102    61.73  39.6321  -84.2675             2           0   
3          103    34.75  39.8839  -75.4669            10           1   
4          104   848.62  48.8856 -103.0098             8           0   
5          105    58.65  40.2878  -74.0162             7           1   
6          106   495.83  41.0767  -74.5982             0           0   
7          107   742.37  43.3526 -102.5411            11           1   
8          108    32.65  38.3880  -79.9906             8           1   
9          109   905.28  39.5803  -87.4616            11           0   
10         110  1084.50  45.6040  -94.1591            11           1   
11         111   109.08  40.3928 -111.7941            11           0   
12         112   827.17  40.4731  -80.9596            12     

C:\Users\Mansi\AppData\Local\Temp\ipykernel_9672\124331289.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  outputofproposed=proposedlogic(testdf[1:2])


0.0

21
    Unnamed: 0      amt      lat      long  category_lbl  gender_lbl  \
0          100   366.72  41.4575  -74.1659             4           1   
1          101    41.46  48.8878 -118.2105             2           0   
2          102    61.73  39.6321  -84.2675             2           0   
3          103    34.75  39.8839  -75.4669            10           1   
4          104   848.62  48.8856 -103.0098             8           0   
5          105    58.65  40.2878  -74.0162             7           1   
6          106   495.83  41.0767  -74.5982             0           0   
7          107   742.37  43.3526 -102.5411            11           1   
8          108    32.65  38.3880  -79.9906             8           1   
9          109   905.28  39.5803  -87.4616            11           0   
10         110  1084.50  45.6040  -94.1591            11           1   
11         111   109.08  40.3928 -111.7941            11           0   
12         112   827.17  40.4731  -80.9596            12     

C:\Users\Mansi\AppData\Local\Temp\ipykernel_9672\124331289.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  outputofproposed=proposedlogic(testdf[1:2])


0.0

In [83]:
if outputofproposed==0.0:
    print("According to proposedlogic No Fraud Detected")
else:
    print("According to proposedlogic Fraud Detected")

According to proposedlogic No Fraud Detected
According to proposedlogic No Fraud Detected


In [84]:
# print(list(lst))
lst = list(nb)

lst1 = [i for i in lst if i==0]
print(len(lst1),"/",len(lst))

8981 / 9961
8981 / 9961


In [86]:
lst=['entertainment', '2500', 'M', 'Grenada', 'IA', '36.555', '85.2588', 'Radiographer, diagnostic']

In [87]:
le1 = LabelEncoder()
le1.fit(mydata3["category"])
le2 = LabelEncoder()
le2.fit(mydata3["gender"])
le3 = LabelEncoder()
le3.fit(mydata3["city"])
le4 = LabelEncoder()
le4.fit(mydata3["state"])
le5 = LabelEncoder()
le5.fit(mydata3["job"])

LabelEncoder()

LabelEncoder()

In [88]:
le1.transform(['entertainment'])[0]

0

0

In [89]:
newlst=[float('2500'),float('36.555'), float('85.2588'),le1.transform(['entertainment'])[0],le2.transform(['M'])[0], le3.transform(['Grenada'])[0], le4.transform(['IA'])[0],le5.transform(['Radiographer, diagnostic'])[0]]
newlst

[2500.0, 36.555, 85.2588, 0, 1, 305, 12, 386]

[2500.0, 36.555, 85.2588, 0, 1, 305, 12, 386]

In [90]:
data = pd.DataFrame(newlst).T
data.columns = ['amt','lat','long','category_lbl','gender_lbl','city_lbl','state_lbl','job_lbl']
data

,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl
0,2500.0,36.555,85.2588,0.0,1.0,305.0,12.0,386.0


,amt,lat,long,category_lbl,gender_lbl,city_lbl,state_lbl,job_lbl
0,2500.0,36.555,85.2588,0.0,1.0,305.0,12.0,386.0


In [91]:
nb=hmm_model.predict(pd.DataFrame(data))
nb[0]

1

1

In [92]:
# count=0
# while True:
#     if count >3:        
#         if count >5:        
#             if count >10:
#                 count=0
#                 print("no if",count) 
#             else:
#                 count+=1
#                 print("third if",count) 
#         else:
#             count+=1
#             print("second if",count)        
#     else:
#         count+=1
#         print("First if",count)  
#     k = 0xff  # Esc for quiting the app
#     if k==27:
#         break